# Red neuronal convolucional (MODELO BASE) para la caracterización del coeficiente de difusión

## 1.- Cargar paqueterías necesarias para la CNN

In [45]:
#------- Cargamos paqueterías necesarias para la CNN ------#
#--------                 INICIO DE LA CNN    -------------#
import numpy as np   # Si encontramos alguna matriz por ahí 
import matplotlib.pyplot as plt   # Para poder graficar los valores que se encuentren por ahí 
import seaborn as sns  # Para poder graficar las estadísticas 
import pandas as pd  # Para poder cargar la base de datos que se va a trabajar 
import tensorflow as tf 
# Importamos paqueterías para poder dividir nuestros datos 
import os, shutil  # os para navegar por directorios y acceder a archivos de imagen, shutil para copiar las imágenes seleccionadas a sus respectivas carpetas  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler  # Para poder normalizar, que en este caso se va a hacer de 0 a 1
from sklearn.model_selection import train_test_split # Para poder dividir las soluciones en train, test y el de evaluación final
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import datasets
from keras import Sequential  # Donde se van a almacenar las capas de la red neuronal 
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from keras.optimizers import SGD 


1.1.- Quitar imágenes que no son imágenes

In [46]:
from PIL import Image
import os

root_dir = "skin224_new"  # Por ejemplo: "skin224"
imagen_contador = 1
imagenes_eliminadas = []

# Extensiones válidas (opcional, pero recomendable)
extensiones_validas = ('.jpg', '.jpeg', '.png', '.bmp')

# Recorre recursivamente todas las carpetas e imágenes
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if not file.lower().endswith(extensiones_validas):
            continue  # Omitir archivos que no son imágenes

        file_path = os.path.join(subdir, file)
        try:
            with Image.open(file_path) as img:
                img.verify()
        except Exception as e:
            print(f"{imagen_contador}. Imagen inválida, será eliminada: {file_path} - {e}")
            try:
                os.remove(file_path)
                imagenes_eliminadas.append(file_path)
                print(f"   ✅ Eliminada: {file_path}")
            except Exception as delete_error:
                print(f"   ❌ Error al eliminar: {delete_error}")
        imagen_contador += 1

print(f"\nTotal de imágenes eliminadas: {len(imagenes_eliminadas)}")


Total de imágenes eliminadas: 0


## 2.- División de la base de datos en 70% train, 20% test y 10% validation 

In [47]:
## Divición de la base de datos, en train, test y validation. 
## Se requiere que los datos divididos tengan un 0.7, 0.2 y 0.1 respectivamente.
## Realizarlo unicamente una vez cuando se programe en una laptop nueva.
import random 
## ----Crear las carpetas donde se van almacenar las carpetas con sus respectivas imágenes
if not os.path.exists("skin_split"):
    os.makedirs("skin_split")
    os.makedirs("train")
    os.makedirs("test")
    os.makedirs("vali")
    shutil.move("vali","./skin_split")
    shutil.move("train","./skin_split")
    shutil.move("test","./skin_split")

# Rutas de trabajo donde se van a guardar las cosas 
ruta_origen = "skin224_new"
ruta_train = "./skin_split/train"
ruta_test = "./skin_split/test"
ruta_vali = "./skin_split/vali"

# Rutas 
# Recorrer la carpeta 
for clase in os.listdir(ruta_origen): 
    if clase in ['train','test','vali']:
        continue

    ruta_clase = os.path.join(ruta_origen,clase)
    if not os.path.isdir(ruta_clase):
       continue

    os.makedirs(os.path.join(ruta_train,clase),exist_ok =True)
    os.makedirs(os.path.join(ruta_test,clase),exist_ok =True)
    os.makedirs(os.path.join(ruta_vali,clase),exist_ok =True)

    imagenes = os.listdir(ruta_clase)

# Dividir los datos de train + val 70% y test 30%
    train_val, test = train_test_split(imagenes, test_size=0.2, shuffle = True)
    train, val = train_test_split(train_val, test_size = 0.125, shuffle=True)
    # Con eso se dividió en 70, 20 y 10 
    # Dividir el train_val 80% a train 70% y val 10% 
# Función para copiar las imágenes a su destino 
    def copiar(imagenes, detino):
        for img in imagenes:
            src = os.path.join(ruta_clase, img)
            dst = os.path.join(detino, clase, img)
            shutil.copy2(src,dst)

    copiar(train,ruta_train)
    copiar(val, ruta_vali)
    copiar(test,ruta_test)

## 3.- Cargar la base de datos de las carpetas para el entrenamiento

In [48]:
# -------- CARGAR LA BASE DE DATOS CON KERAS  ---------- #
# -------- ImageGenerator     --------------------------#
datagen = ImageDataGenerator(rescale = 1./255)   # De una vez se puede normalizar los valores de las imagenes 
datagen_train = ImageDataGenerator(  # Vi que había un problema con la data, así que cree más imagenes 
    rescale = 1./255,   # Escala 
    rotation_range = 20,  # Aplica rotaciones aleatorias de hasta 20 grados 
    zoom_range = 0.2,  # Aplica zoom de hasta el 20% 

    # LO SIGUIENTE CONSIDERO QUE NO ES NECESARIO (OPINIÓN DE ASESORES ? )
   # width_shift_range = 0.1,# Desplaza horizontalmente las imágenes un 10%
   # height_shift_range = 0.1, # Desplaza verticalmente las imágenes un 10%
   # horizontal_flip=True
)
# Iterador para cargar los subdirectorios que contienen las imágenes de test 20%
test_it = datagen.flow_from_directory(
    directory = "skin_split/test",  # La ruta donde se encuentran las Imágenes 
    class_mode = "categorical",  # Especificar la tarea que corresponde a multiclase
    batch_size = 16,   # Durante el entrenamiento dara un conjunto de 32 imágenes aleatorias 
    shuffle = True,   # Aleatoriedad robusta 
    target_size=(224, 224)
)
# Iterador para cargar los subdirectorios que contienen las imágenes de train 70%
train_it = datagen_train.flow_from_directory(
    directory = "skin_split/train",  # La ruta donde se encuentran las Imágenes 
    class_mode = "categorical",  # Especificar la tarea que corresponde a multiclase
    batch_size = 16,   # Durante el entrenamiento dara un conjunto de 32 imágenes aleatorias 
    shuffle = True,   # Aleatoriedad robusta 
    target_size=(224, 224)
)
# Iterador para cargar los subdirectorios que contienen las imágenes de validation 10%
vali_it = datagen.flow_from_directory(
    directory = "skin_split/vali",  # La ruta donde se encuentran las Imágenes 
    class_mode = "categorical",  # Especificar la tarea que corresponde a multiclase
    batch_size = 16,   # Durante el entrenamiento dara un conjunto de 32 imágenes aleatorias 
    shuffle = True,   # Aleatoriedad robusta
    target_size=(224, 224) 
)

Found 26000 images belonging to 500 classes.
Found 91000 images belonging to 500 classes.
Found 13000 images belonging to 500 classes.


## 4.- Construcción de la red neuronal propia basada en la teoría 

In [49]:
## Construcción de la red neuronal CNN-VGG16
## Una red neuronal CNN-VGG16 admite imágenes de la forma (224,224,3) para este trabajo, se utilizaron estas medidas. 

# Llamamos la clase secuencial donde se anidaran las capas 
model = Sequential()
# ----------Primera capa de convolución
model.add(Conv2D(filters = 32, 
                 kernel_size = (3,3), 
                 padding = "same", 
                 activation='relu', 
                 input_shape=(224,224,3)))
# Primera capa de pooling 
model.add(MaxPooling2D(pool_size=(2,2)))

# ----------Segunda capa de convolución
model.add(Conv2D(filters = 64, 
                 kernel_size=(3,3), 
                 padding = "same",
                 activation='relu'))
# Segunda capa de pooling
model.add(MaxPooling2D(pool_size=(2,2)))

# -----------Tercera capa de convolución 
model.add(Conv2D(filters = 64,
                 kernel_size= (3,3),
                 padding = "same",
                 activation = "relu"))

model.add(MaxPooling2D(pool_size=(2,2)))

# -----------Cuarta capa de convolución 
model.add(Conv2D(filters = 64,
                 kernel_size= (3,3),
                 padding = "same",
                 activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_it.num_classes, activation='softmax'))
model.summary()

## ------Aplanamos las mejores características 
#model.add(Flatten())

## --------Agregamos la red neuronal densa 
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))  # Para que se prendan 50% de las neuronas
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))  # Para que se prendan 50% de las neuronas
## La última capa que me dará las características, usamos un softmax
#model.add(Dense(train_it.num_classes, activation = 'softmax'))

## Mostramos como va nuestra red neuronal CNN
#model.summary()

/home/letmitaf/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_38 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_39 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1024)           │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 500)            │       512,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 672,308 (2.56 MB)

 Trainable params: 672,308 (2.56 MB)

 Non-trainable params: 0 (0.00 B)

## 5.- Compilación de la red neuronal 

In [50]:
# Compilamos la red neuronal, 
# Se elige un optimizador de adam, la función de perdida
# categorical_crossentropy porque se tienen multiples clases 
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics = ["accuracy"]
)

## 6.- Entrenamiento del modelo 

In [51]:
history = model.fit(
    train_it,
    epochs = 10,
    batch_size = 100,
    validation_data = vali_it,
    verbose = True
)
# Graficas de error 
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label = 'Pérdida en el entrenamiento')
plt.plot(history.history['val_loss'],label = 'Pérdida de validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

# Grafica de aprendizaje 
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'],label='Presición en el entrenamiento')
plt.plot(history.history['val_accuracy'],label = 'Precisión en la validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Época')
plt.xlabel('Precisión')
plt.legend()
plt.tight_layout()
plt.show()


Epoch 1/10


/home/letmitaf/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 106/5688 ━━━━━━━━━━━━━━━━━━━━ 7:21:03 5s/step - accuracy: 0.0098 - loss: 6.0057

KeyboardInterrupt: 